<a href="https://colab.research.google.com/github/POOJITHAPEDDAREDDY/pthon_proj/blob/master/project_queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=ac81faa088a4da32233b7003b2c0d879e56f3a4451d8fd873315ba599e226a8b
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


In [3]:
spark = SparkSession.builder.appName("e-commercedata").getOrCreate()

In [8]:
df=spark.read.csv('final_data.csv',header=True,inferSchema=True)

In [19]:
#Sushant needs to know whether his product named Hair dryer is in his regional Datawarehouse Portugal
#and display all the products in the Portugal datawarehouse

portugal_df = df.filter(df.Warehouse == "Portugal")
dryer_portugal = portugal_df.filter(portugal_df.product_name == "Hair dryer").count() > 0
dryer_portugal







False

In [33]:
#An Employee at the company was asked to display the total revenue generated by the cosmetics category
#and display the product name with the highest rating along with the brand name.

cosmetics_df = df.filter(col("Category") == "cosmetics")
cosmetics_df.show()
total_revenue = cosmetics_df.agg(sum("Price").alias("TotalRevenue")).collect()[0][0]
total_revenue
highest_rated_product = cosmetics_df.orderBy(col("Rating").desc()).select("product_name", "Brand", "Rating").first()
highest_rated_product


+----------+--------------------+-------+--------------------+---------+-------------+--------------------+------+--------------------+
|product_id|        product_name|  Price|         Description| Category|StockQuantity|               Brand|Rating|           Warehouse|
+----------+--------------------+-------+--------------------+---------+-------------+--------------------+------+--------------------+
|   10134.0|            Lip Balm|3539.17|Outside contain d...|cosmetics|         42.0|         Urban Decay|   2.5|             Vietnam|
|    4066.0|          Lip Primer|4755.07|Suffer trade betw...|cosmetics|         71.0|Anastasia Beverly...|   3.4|            Colombia|
|    6833.0|         body lotion|2707.52|Baby sign arrive ...|cosmetics|         33.0|   Benefit Cosmetics|   4.6|United Arab Emirates|
|    4637.0|      Eyebrow Pomade|2746.45|Nothing style pay...|cosmetics|         72.0|         Huda Beauty|   1.4|         South Korea|
|    6508.0|              primer|3128.62|Minute 

Row(product_name='Eye Mask', Brand='NARS', Rating=5.0)

In [36]:
#The manager at the warehouse was informed that the stock is below the minimum quantity and was told to restock it.
#Restock every category , every product that is less than 15(stock quantity)

categories_to_restock = ["Cosmetics", "Books", "Accessories", "Footwear", "Clothing", "Furniture"]
restock_df = df.filter((col("Category").isin(categories_to_restock)) & (col("StockQuantity") < 15))
restock_df.show()
restock_level = 50
restocked_df = restock_df.withColumn("StockQuantity", col("StockQuantity") + restock_level)
restocked_df.show()

+----------+-------------+------+--------------------+---------+-------------+--------------------+------+--------------------+
|product_id| product_name| Price|         Description| Category|StockQuantity|               Brand|Rating|           Warehouse|
+----------+-------------+------+--------------------+---------+-------------+--------------------+------+--------------------+
|    5135.0|Console Table| 848.5|New perform box t...|Furniture|         13.0|   Century Furniture|   4.0|              Greece|
|    9739.0|     Recliner|604.26|Eat deep down mac...|Furniture|          8.0|      Crate & Barrel|   2.6|United Arab Emirates|
|    6850.0|   Nightstand|175.31|Do land off apply...|Furniture|         12.0|     Williams-Sonoma|   1.5|           Indonesia|
|    1641.0|       Vanity| 709.3|Seek too read inc...|Furniture|          3.0|    American Leather|   4.6|             Finland|
|     955.0| Office Chair|686.74|Without history f...|Furniture|         13.0|    American Leather|   2.

In [44]:
#supervisor needs the list of products that are stored in Hungary warehouse and Kuwait warehouse
#and display which warehouse has the highest stock quantity.

filtered_df = df.filter(col("Warehouse").isin(["Hungary", "Kuwait"]))
filtered_df.show()
warehouse_stock = filtered_df.groupBy("Warehouse").agg(sum("StockQuantity").alias("TotalStockQuantity"))
warehouse_stock.show()
highest_stock_warehouse = warehouse_stock.orderBy(col("TotalStockQuantity").desc()).first()[0]
highest_stock_warehouse

+----------+------------------+------+--------------------+--------+-------------+------------+------+---------+
|product_id|      product_name| Price|         Description|Category|StockQuantity|       Brand|Rating|Warehouse|
+----------+------------------+------+--------------------+--------+-------------+------------+------+---------+
|    2928.0|      Toy Tool Set| 97.19|Kid future itself...|    Toys|         49.0|      Disney|   1.1|  Hungary|
|   10209.0|         Toy Train|337.68| Myself strong just.|    Toys|         42.0|       VTech|   2.1|  Hungary|
|    5230.0|           Toy Car| 16.18|Lay stay pattern ...|    Toys|         48.0|Transformers|   4.3|   Kuwait|
|    5387.0|         Robot Toy|220.98|Story receive des...|    Toys|         90.0|   Playmobil|   2.4|   Kuwait|
|    3241.0|      Toy Tool Set|132.42|Federal clearly g...|    Toys|         80.0|Fisher-Price|   4.5|  Hungary|
|    8736.0|              Lego|366.93|Final effort trea...|    Toys|         76.0| Nickelodeon| 

'Hungary'

In [76]:
updated_df = df
# updated_df = updated_df.filter(col("product_name") == "Eye Cream").withColumn("Price", lit(600.00))
updated_df = updated_df.withColumn("Price", when(col("product_name") == "Eye Cream", lit(600.00)).otherwise(col("Price")))
updated_df.show()

+----------+--------------+------+--------------------+--------+-------------+----------------+------+--------------------+
|product_id|  product_name| Price|         Description|Category|StockQuantity|           Brand|Rating|           Warehouse|
+----------+--------------+------+--------------------+--------+-------------+----------------+------+--------------------+
|    2928.0|  Toy Tool Set| 97.19|Kid future itself...|    Toys|         49.0|          Disney|   1.1|             Hungary|
|    5349.0|  Rubik's Cube| 344.7|Range heavy thoug...|    Toys|         26.0|          Disney|   4.6|             Vietnam|
|     847.0|     Dollhouse| 80.21|Form fill yet nat...|    Toys|         70.0|Thomas & Friends|   2.8|           Singapore|
|    3463.0|  Toy Dinosaur|442.73|Low matter build ...|    Toys|         85.0|          Disney|   1.7|           Indonesia|
|    9252.0|  Rubik's Cube|182.74|Take walk step de...|    Toys|         82.0|       DC Comics|   2.3|             Ireland|
|     26

In [77]:
updated_df.filter(col("product_name") == "Eye Cream").show()

+----------+------------+-----+--------------------+---------+-------------+--------------------+------+------------+
|product_id|product_name|Price|         Description| Category|StockQuantity|               Brand|Rating|   Warehouse|
+----------+------------+-----+--------------------+---------+-------------+--------------------+------+------------+
|    9032.0|   Eye Cream|600.0|Republican laugh ...|cosmetics|         46.0|    E.l.f. Cosmetics|   2.8| New Zealand|
|    5559.0|   Eye Cream|600.0|Item he fight eff...|cosmetics|         89.0|        Estée Lauder|   3.1|     Vietnam|
|    3059.0|   Eye Cream|600.0|International tha...|cosmetics|         19.0|              Morphe|   2.7|      Poland|
|    1036.0|   Eye Cream|600.0|A follow populati...|cosmetics|         38.0|               Tarte|   4.3|       Italy|
|   11991.0|   Eye Cream|600.0|Around market poi...|cosmetics|         15.0|        Fenty Beauty|   3.5|      Brazil|
|    1713.0|   Eye Cream|600.0|Discover recently...|cosm

In [78]:
#employee needs to know whether product name dollhouse is in the stock and wants to display the stockQuantity of dollhouse across different warehouses
updated_df.filter(col("product_name") == "Dollhouse").show()

+----------+------------+------+--------------------+--------+-------------+----------------+------+--------------------+
|product_id|product_name| Price|         Description|Category|StockQuantity|           Brand|Rating|           Warehouse|
+----------+------------+------+--------------------+--------+-------------+----------------+------+--------------------+
|     847.0|   Dollhouse| 80.21|Form fill yet nat...|    Toys|         70.0|Thomas & Friends|   2.8|           Singapore|
|    1197.0|   Dollhouse|400.68|Me where vote tru...|    Toys|         39.0|       Star Wars|   3.2|            Malaysia|
|    5087.0|   Dollhouse|115.79|Drop look arrive ...|    Toys|         24.0|            Lego|   4.2|              France|
|     223.0|   Dollhouse|469.83|Behind range book...|    Toys|         57.0|    Fisher-Price|   2.8|               Spain|
|     913.0|   Dollhouse|387.42|Where read rock s...|    Toys|         20.0|    Little Tikes|   4.3|              Russia|
|    9009.0|   Dollhouse

In [ ]:
#
